## Infer

In [12]:
import torch
import torch.nn as nn

In [13]:
class Net(nn.Module):
    """Network inference
    """
    def __init__(self, path = 'test/epoch21-0.8867.pth'):
        super().__init__()
        self.i_proj = nn.Linear(8, 64, False)
        self.alpha = nn.Parameter(torch.rand(1, 64))
        self.h0 = nn.Parameter(torch.rand(1, 64))
        self.o_proj = nn.Linear(64, 1, False)
        self.act = nn.ReLU6()
        self.load(path)

    @torch.no_grad()
    def load(self, path):
        ckpt = torch.load(path)

        self.i_proj.weight.copy_(ckpt['0.weight'])
        self.alpha.copy_(ckpt['3.alpha'].squeeze(2).transpose(0, 1))
        self.h0.copy_(ckpt['3.x0'] * (1 - self.alpha))
        self.o_proj.weight.copy_(ckpt['6.weight'])

    @torch.no_grad()
    def forward(self, x, hx):
        g = self.alpha
        act = self.act
        i = act(self.i_proj(x))
        # h = g * hx + (1 - g) * i
        h = i + g * (hx - i)
        y = self.o_proj(act(h))
        y = torch.sigmoid(y).squeeze(1)
        return y, h


In [14]:
net = Net()
net.eval()

Net(
  (i_proj): Linear(in_features=8, out_features=64, bias=False)
  (o_proj): Linear(in_features=64, out_features=1, bias=False)
  (act): ReLU6()
)

In [15]:
x = torch.tensor(
    [[4.882802, 0.4116477, 0.2582662, 1.1432298, 0.34995735, 0.15397726, 0.07350865, 0.]]
)
h = net.h0
y, h = net(x, h)
y

tensor([0.9168])

In [16]:
x = torch.tensor(
    [[3.9512436 , 0.4121284 , 0.57202524, 0.9942352 , 0.38036877, 0.11640994, 0.07208236, 0.08725858]]
)
y, h = net(x, h)
y

tensor([0.9293])

## ONNX

In [21]:
import onnx
import numpy as np
import pandas as pd
import onnxruntime as ort

In [22]:
x = torch.tensor(
    [[4.882802, 0.4116477, 0.2582662, 1.1432298, 0.34995735, 0.15397726, 0.07350865, 0.]]
)

In [23]:
dummy = (x, net.h0)

torch.onnx.export(
    net,
    dummy, 'test/model.onnx',
    input_names=['input', 'hx'],
    output_names=['output', 'h'],
    opset_version=13
)


C:\Users\pontosense\AppData\Local\Temp\ipykernel_28832\4284259381.py:3: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


In [25]:
sess = ort.InferenceSession('test/model.onnx', providers=["CPUExecutionProvider"])
out = sess.run(None, {'input': x.numpy(), 'hx': net.h0.detach().numpy()})

In [26]:
net(*dummy)

(tensor([0.9168]),
 tensor([[7.8661e-01, 2.1986e-01, 6.5865e-01, 1.1748e+00, 3.1769e-03, 5.1610e-01,
          4.0148e-02, 4.7996e-01, 5.2421e-02, 4.8228e-02, 8.8668e-02, 8.6797e-02,
          1.9795e+00, 1.0100e+00, 1.2751e-01, 7.0789e-02, 1.2766e+00, 1.2876e+00,
          1.1415e-01, 1.0223e+00, 4.6984e-01, 1.6167e-01, 7.9335e-03, 4.8726e-01,
          1.5820e+00, 4.6978e-01, 1.7345e+00, 6.5467e-01, 4.1932e-03, 2.1408e-01,
          9.3604e-02, 1.6803e-01, 4.5541e-02, 3.7650e-02, 7.2401e-01, 3.2143e-01,
          1.9473e+00, 1.5760e+00, 1.0006e+00, 1.1182e+00, 4.7248e-01, 3.9661e-03,
          3.0456e-01, 6.2650e-01, 3.0756e-01, 1.8634e-03, 2.7375e-01, 5.0396e-02,
          5.7525e-01, 1.0706e+00, 2.9057e-01, 2.5634e-01, 1.4951e-01, 7.1377e-01,
          6.0333e-02, 6.9569e-01, 9.0938e-01, 1.5414e-01, 2.4672e-02, 1.0866e-02,
          7.2793e-02, 8.1577e-01, 1.1750e+00, 5.2371e-02]]))

In [27]:
out

[array([0.9167516], dtype=float32),
 array([[7.8660810e-01, 2.1985634e-01, 6.5865248e-01, 1.1747572e+00,
         3.1768831e-03, 5.1610315e-01, 4.0148247e-02, 4.7995839e-01,
         5.2421257e-02, 4.8227832e-02, 8.8668346e-02, 8.6796865e-02,
         1.9795123e+00, 1.0099574e+00, 1.2751462e-01, 7.0789427e-02,
         1.2765919e+00, 1.2876171e+00, 1.1414590e-01, 1.0222778e+00,
         4.6984407e-01, 1.6166882e-01, 7.9335170e-03, 4.8725992e-01,
         1.5819535e+00, 4.6977788e-01, 1.7345171e+00, 6.5467143e-01,
         4.1932315e-03, 2.1407516e-01, 9.3604356e-02, 1.6803156e-01,
         4.5540698e-02, 3.7650466e-02, 7.2400790e-01, 3.2143021e-01,
         1.9473239e+00, 1.5759907e+00, 1.0006256e+00, 1.1182337e+00,
         4.7248182e-01, 3.9660963e-03, 3.0456409e-01, 6.2650490e-01,
         3.0755606e-01, 1.8633652e-03, 2.7375156e-01, 5.0396293e-02,
         5.7524836e-01, 1.0705720e+00, 2.9056746e-01, 2.5634307e-01,
         1.4951283e-01, 7.1377492e-01, 6.0333371e-02, 6.9569373e-01

## ESP-DL

In [28]:
from torch.utils.data import (
    Dataset,
    DataLoader
)
from esp_ppq.api import (
    espdl_quantize_torch,
    espdl_quantize_onnx
)
from esp_ppq.executor.torch import TorchExecutor


    ___________ ____        ____  ____  ____
   / ____/ ___// __ \      / __ \/ __ \/ __ \
  / __/  \__ \/ /_/ /_____/ /_/ / /_/ / / / /
 / /___ ___/ / ____/_____/ ____/ ____/ /_/ /
/_____//____/_/         /_/   /_/    \___\_\




In [29]:
from utils import (
    process_label,
    process_feature
)

In [30]:
data = []
label = []
for path in [
    'ATR_GN_P4-250120-00138-2025-09-04-15-372127-373946.csv',
    'ATR_GN_P4-250120-00138-2025-09-07-20-3155534-3159779.csv'
    ]:
    path = f'dataset/train/{path}'
    df = pd.read_csv(path, index_col=0)

    process_label(df, path)
    process_feature(df)

    df = df[
        ['ln_pts', 'z_iqr', 'z_std', 'z_c', 'range', 'l_r', 's_r', 'dist', 'label']
    ].astype('float32').values
    h = net.h0.detach()
    for x in df:
        data.append((x[:-1], h.numpy()[0]))
        label.append(x[-1])
        x = torch.from_numpy(x[:-1]).unsqueeze(0)
        y, h = net(x, h)


In [31]:
class QuantDataset(Dataset):

    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.label[idx]
        return data, label


In [32]:
ds = QuantDataset(data, label)
dl = DataLoader(ds, batch_size=32, shuffle=False)

def collate_fn(batch):
    return batch[0]

In [33]:
graph = espdl_quantize_onnx(
    onnx_import_file='test/model.onnx',
    espdl_export_file='test2/model.espdl',
    calib_dataloader=dl,
    calib_steps=32,
    input_shape=[[1, 8], [1, 64]],  # 输入形状，批次为 1
    inputs=None,
    target="esp32s3",  # 量化目标类型
    num_of_bits=16,  # 量化位数
    collate_fn=collate_fn,
    device='cpu',
    error_report=True,
    skip_export=False,
    export_test_values=True,
    verbose=2
)

[16:32:25] PPQ Quantization Fusion Pass Running ...       Finished.
[16:32:26] PPQ Quantize Simplify Pass Running ...         Finished.
[16:32:26] PPQ Parameter Quantization Pass Running ...    Finished.
[16:32:26] PPQ Runtime Calibration Pass Running ...       

Calibration Progress(Phase 2): 100%|██████████| 32/32 [00:00<00:00, 39.72it/s]


Finished.
[16:32:27] PPQ Quantization Alignment Pass Running ...    Finished.
[16:32:27] PPQ Passive Parameter Quantization Running ... Finished.
--------- Network Snapshot ---------
Num of Op:                    [9]
Num of Quantized Op:          [9]
Num of Variable:              [19]
Num of Quantized Var:         [19]
------- Quantization Snapshot ------
Num of Quant Config:          [28]
ACTIVATED:                    [9]
OVERLAPPED:                   [9]
PASSIVE:                      [9]
FP32:                         [1]
Network Quantization Finished.


Analysing Graphwise Quantization Error(Phrase 1):: 100%|██████████| 8/8 [00:00<00:00, 185.81it/s]
Analysing Graphwise Quantization Error(Phrase 2):: 100%|██████████| 8/8 [00:00<00:00, 195.07it/s]


Layer           | NOISE:SIGNAL POWER RATIO 
/i_proj/MatMul: | ████████████████████ | 1.447%
/o_proj/MatMul: |                      | 0.439%


Analysing Layerwise quantization error:: 100%|██████████| 2/2 [00:00<00:00, 90.74it/s]

Layer           | NOISE:SIGNAL POWER RATIO 
/i_proj/MatMul: | ████████████████████ | 0.322%
/o_proj/MatMul: |                      | 0.000%
[INFO][ESPDL][2025-10-10 16:32:27]:  Skip /Constant_output_0 because it's not exportable
[INFO][ESPDL][2025-10-10 16:32:27]:  Skip /Constant_output_0 because it's not exportable
[INFO][ESPDL][2025-10-10 16:32:27]:  Skip /Constant_output_0 because it's not exportable
[INFO][ESPDL][2025-10-10 16:32:27]:  skip not QuantableOperation
[INFO][ESPDL][2025-10-10 16:32:27]:  Skip /Constant_output_0 because it's not exportable


In [34]:
criterion = torch.nn.BCEWithLogitsLoss()

loss = 0
err = 0
for batch, label in dl:
    y_pred, h = net(*batch)
    loss += criterion(y_pred, label)
    err += ((y_pred>0.8) != label).sum()

loss /= len(dl)
print(f"origin model loss: {loss.item():.5f}")

print(f"origin model err: {err/len(ds):.5f}")

origin model loss: 0.81184
origin model err: 0.10344


In [35]:
executor = TorchExecutor(graph=graph, device='cpu')
loss = 0
err = 0
for batch, label in dl:
    y_pred, h = executor(batch)
    loss += criterion(y_pred, label)
    err += ((y_pred>0.8) != label).sum()

loss /= len(dl)
print(f"quant model loss: {loss.item():.5f}")

print(f"quant model err: {err/len(ds):.5f}")

quant model loss: 0.81254
quant model err: 0.10901


In [36]:
executor = TorchExecutor(graph=graph, device='cpu')
x = torch.tensor(
    [[4.882802, 0.4116477, 0.2582662, 1.1432298, 0.34995735, 0.15397726, 0.07350865, 0.]]
)
h = net.h0.detach()
batch = [x, h]
y_pred = executor(batch)


In [37]:
y_pred

[tensor([0.9008]),
 tensor([[0.6885, 0.2198, 0.5027, 1.0769, 0.0032, 0.4906, 0.0402, 0.3770, 0.0524,
          0.0482, 0.0773, 0.0868, 1.7412, 0.9561, 0.1274, 0.0708, 1.0892, 1.1113,
          0.1141, 0.8634, 0.5474, 0.1617, 0.0079, 0.4052, 1.3741, 0.4323, 1.5175,
          0.5607, 0.0044, 0.2141, 0.0936, 0.1659, 0.0455, 0.0331, 0.6772, 0.3899,
          1.6473, 1.3566, 0.8929, 0.9310, 0.3630, 0.0040, 0.2827, 0.5604, 0.2706,
          0.0018, 0.3129, 0.0504, 0.4323, 0.9281, 0.2184, 0.2098, 0.1421, 0.6945,
          0.0536, 0.5927, 0.8116, 0.1542, 0.0247, 0.0109, 0.0692, 0.7612, 1.0388,
          0.0524]])]